---
## 🎉 EXPERIMENT COMPLETE!

### 📊 Sonuçlar:

Tüm sonuçlar `./results/` klasöründe:
- `final_results_<run_id>.csv` - CSV formatında
- `final_results_<run_id>.json` - JSON formatında  
- `final_results_<run_id>.xlsx` - Excel formatında
- `comparison_plot_<run_id>.png` - Karşılaştırma grafiği
- `config_<run_id>.json` - Experiment konfigürasyonu

### 🎯 Test Edilen Modeller:

1. **Baseline RF** - Class-weighted Random Forest
2. **KAN Base** - Lightweight KAN (Focal Loss)
3. **KAN + Attention** - Feature-level attention (ÖZGÜN KATKI)

### 📈 Ana Metrikler:

- **Recall** (0.80+ hedef) - Defect detection için kritik
- **F2 Score** (0.65+ hedef) - Recall'a ağırlık veren metric
- **Precision** - False positive kontrolü
- **Accuracy** - Genel performans

### ✅ Başarı Kriterleri:

- ✓ Recall korundu (0.80+)
- ✓ F2 optimize edildi
- ✓ Hafif model (CPU friendly)
- ✓ Leakage yok (test seti izole)
- ✓ SMOTE sadece train'de

---

**🎓 Özgün Katkı:** Feature-level attention mechanism - Her sample için feature'lara dinamik ağırlık vererek model interpretability ve performance artışı sağlandı!

In [ ]:
# ============================================================================# COMPILE ALL RESULTS# ============================================================================# Check if all required variables existrequired_vars = {    'CONFIG': 'Run the Config cell first!',    'prepared_datasets': 'Run the Dataset Preparation cell first!',    'baseline_results': 'Run the Baseline RF training cell first!',    'kan_base_results': 'Run the KAN Base training cell first!',    'kan_attention_results': 'Run the KAN+Attention training cell first!'}missing_vars = []for var_name, message in required_vars.items():    if var_name not in dir():        missing_vars.append(f"❌ {var_name}: {message}")if missing_vars:    print("\n" + "="*70)    print("⚠️  MISSING REQUIRED VARIABLES")    print("="*70)    print("\nPlease run these cells IN ORDER before running this cell:\n")    for msg in missing_vars:        print(f"   {msg}")    print("\n" + "="*70)    print("\n📌 Quick Guide:")    print("   1. Mount Google Drive (cell at top)")    print("   2. Run Config cell")    print("   3. Run Dependencies cell")    print("   4. Run Dataset Loading cell")    print("   5. Run Data Preparation cell")    print("   6. Run SMOTE cell")    print("   7. Run Baseline RF cell")    print("   8. Run KAN Base cell")    print("   9. Run KAN+Attention cell")    print("   10. NOW run this cell!")    print("="*70)    raise RuntimeError("Missing required variables - see messages above")print("\n" + "="*70)print("📊 COMPILING FINAL RESULTS")print("="*70)# Organize all resultsall_results = {}for dataset_name in CONFIG['datasets']:    if dataset_name not in prepared_datasets:        continue        all_results[dataset_name] = {        'dataset_info': {            'n_samples': len(prepared_datasets[dataset_name]['y_train']) +                         len(prepared_datasets[dataset_name]['y_val']) +                         len(prepared_datasets[dataset_name]['y_test']),            'n_features': prepared_datasets[dataset_name]['n_features'],            'defect_ratio': np.mean(np.concatenate([                prepared_datasets[dataset_name]['y_train'],                prepared_datasets[dataset_name]['y_val'],                prepared_datasets[dataset_name]['y_test']            ]))        },        'Baseline_RF': baseline_results[dataset_name],        'KAN_Base': kan_base_results[dataset_name],        'KAN_Attention': kan_attention_results[dataset_name]    }# Create summary dataframeresults_df = compile_final_results(all_results, CONFIG)# Print summaryprint_final_summary(results_df)# Export resultsprint(f"\n{'='*70}")print(f"💾 EXPORTING RESULTS")print(f"{'='*70}\n")export_results(results_df, CONFIG)# Create comparison plotprint(f"\n{'='*70}")print(f"📈 CREATING COMPARISON PLOTS")print(f"{'='*70}\n")create_comparison_plot(results_df, CONFIG)print(f"\n{'='*70}")print(f"✅ ALL RESULTS COMPILED AND EXPORTED")print(f"{'='*70}")

---
## 📊 EXECUTION: Final Results Compilation & Export

Tüm sonuçları derle, karşılaştır ve export et (CSV/JSON/XLSX)

In [ ]:
# ============================================================================
# LOSS ABLATION: Focal vs Weighted BCE (on JM1 only for speed)
# ============================================================================

print("\n" + "="*70)
print("🔬 LOSS ABLATION STUDY - Focal vs Weighted BCE")
print("="*70)
print("📌 Testing on JM1 only (kaynak tasarrufu)\n")

loss_ablation_results = {}

# Test only on JM1 to save resources
dataset_name = 'JM1'
data = prepared_datasets[dataset_name]

for loss_type in ['Focal', 'WeightedBCE']:
    print(f"\n{'='*70}")
    print(f"📊 Loss Type: {loss_type}")
    print(f"{'='*70}")
    
    # Create model
    model = KAN(
        input_dim=data['n_features'],
        hidden_dim=KAN_LITE_CONFIG['hidden_dim'],
        grid_size=KAN_LITE_CONFIG['grid_size'],
        spline_order=KAN_LITE_CONFIG['spline_order'],
        dropout=0.3
    )
    
    # Train
    model, history = train_kan_model(
        model,
        data['X_train_smote'],
        data['y_train_smote'],
        data['X_val'],
        data['y_val'],
        learning_rate=KAN_LITE_CONFIG['learning_rate'],
        epochs=KAN_LITE_CONFIG['epochs'],
        batch_size=KAN_LITE_CONFIG['batch_size'],
        loss_type=loss_type,
        patience=KAN_LITE_CONFIG['patience'],
        verbose=True
    )
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        X_test_t = torch.FloatTensor(data['X_test']).to(device)
        y_test_proba = model(X_test_t).cpu().numpy().flatten()
    
    # Use threshold from Focal loss run (fair comparison)
    optimal_threshold = kan_base_results[dataset_name]['optimal_threshold']
    
    y_test_pred = (y_test_proba >= optimal_threshold).astype(int)
    test_metrics = calculate_all_metrics(data['y_test'], y_test_pred, y_test_proba)
    
    print(f"\n   📊 Test Performance (threshold={optimal_threshold:.2f}):")
    print(f"      Recall:    {test_metrics['recall']:.4f}")
    print(f"      Precision: {test_metrics['precision']:.4f}")
    print(f"      F2:        {test_metrics['f2']:.4f}")
    print(f"      Accuracy:  {test_metrics['accuracy']:.4f}")
    
    loss_ablation_results[loss_type] = {
        'model': model,
        'test_metrics': test_metrics,
        'history': history
    }

# Compare losses
print(f"\n{'='*70}")
print(f"🏆 LOSS COMPARISON (JM1 Dataset)")
print(f"{'='*70}")

focal_f2 = loss_ablation_results['Focal']['test_metrics']['f2']
wbce_f2 = loss_ablation_results['WeightedBCE']['test_metrics']['f2']

focal_recall = loss_ablation_results['Focal']['test_metrics']['recall']
wbce_recall = loss_ablation_results['WeightedBCE']['test_metrics']['recall']

print(f"\n📊 F2 Score:")
print(f"   Focal Loss:       {focal_f2:.4f}")
print(f"   Weighted BCE:     {wbce_f2:.4f}")
print(f"   Winner:           {'Focal' if focal_f2 > wbce_f2 else 'WeightedBCE'} (+{abs(focal_f2-wbce_f2):.4f})")

print(f"\n📊 Recall (Defect Detection):")
print(f"   Focal Loss:       {focal_recall:.4f}")
print(f"   Weighted BCE:     {wbce_recall:.4f}")
print(f"   Winner:           {'Focal' if focal_recall > wbce_recall else 'WeightedBCE'} (+{abs(focal_recall-wbce_recall):.4f})")

print(f"\n✅ LOSS ABLATION COMPLETE")
print(f"{'='*70}")

---
## 🔬 EXECUTION: Loss Ablation Study

Focal Loss vs Weighted BCE karşılaştırması (hızlı)

In [ ]:
# ============================================================================
# KAN + FEATURE ATTENTION - LIGHTWEIGHT (ÖZGÜN KATKI)
# ============================================================================

print("\n" + "="*70)
print("🌟 KAN + FEATURE ATTENTION MODELS (Focal Loss)")
print("="*70)
print("✨ ÖZGÜN KATKI: Sample-specific feature weighting\n")

kan_attention_results = {}

for dataset_name, data in prepared_datasets.items():
    print(f"\n{'='*70}")
    print(f"📊 Dataset: {dataset_name}")
    print(f"{'='*70}")
    
    # Create KAN with Attention
    model = KAN_WithAttention(
        input_dim=data['n_features'],
        hidden_dim=KAN_LITE_CONFIG['hidden_dim'],
        grid_size=KAN_LITE_CONFIG['grid_size'],
        spline_order=KAN_LITE_CONFIG['spline_order'],
        dropout=0.3,
        attention_dim=CONFIG['attention_dim']
    )
    
    # Train
    model, history = train_kan_model(
        model,
        data['X_train_smote'],
        data['y_train_smote'],
        data['X_val'],
        data['y_val'],
        learning_rate=KAN_LITE_CONFIG['learning_rate'],
        epochs=KAN_LITE_CONFIG['epochs'],
        batch_size=KAN_LITE_CONFIG['batch_size'],
        loss_type=KAN_LITE_CONFIG['loss_type'],
        patience=KAN_LITE_CONFIG['patience'],
        verbose=True
    )
    
    # Evaluate on validation
    model.eval()
    with torch.no_grad():
        X_val_t = torch.FloatTensor(data['X_val']).to(device)
        y_val_proba = model(X_val_t).cpu().numpy().flatten()
    
    # Find optimal threshold
    print(f"\n🔍 Finding optimal threshold on validation set...")
    threshold_result = find_optimal_threshold(
        data['y_val'],
        y_val_proba,
        metric='f2',
        threshold_range=CONFIG['threshold_range'],
        step=CONFIG['threshold_step'],
        accuracy_floor=CONFIG['gwo_accuracy_floor'],
        verbose=True
    )
    
    optimal_threshold = threshold_result['threshold']
    
    # Evaluate on test
    print(f"\n🧪 Evaluating on TEST set with threshold={optimal_threshold:.2f}...")
    with torch.no_grad():
        X_test_t = torch.FloatTensor(data['X_test']).to(device)
        y_test_proba = model(X_test_t).cpu().numpy().flatten()
    
    y_test_pred = (y_test_proba >= optimal_threshold).astype(int)
    test_metrics = calculate_all_metrics(data['y_test'], y_test_pred, y_test_proba)
    
    print(f"\n   📊 Test Performance:")
    print_metrics(test_metrics, prefix="   ")
    
    # Extract attention weights for interpretation
    print(f"\n   🔍 Analyzing Feature Attention Weights...")
    attention_weights = model.get_attention_weights(data['X_test'][:10])  # Sample 10 examples
    avg_attention = attention_weights.mean(axis=0)
    top_features_idx = np.argsort(avg_attention)[-5:][::-1]
    
    print(f"      Top 5 Most Attended Features:")
    for i, idx in enumerate(top_features_idx, 1):
        print(f"         {i}. Feature {idx}: {avg_attention[idx]:.4f}")
    
    # Store results
    kan_attention_results[dataset_name] = {
        'model': model,
        'optimal_threshold': optimal_threshold,
        'val_metrics': threshold_result['metrics'],
        'test_metrics': test_metrics,
        'history': history,
        'attention_weights': attention_weights
    }
    
    print(f"\n✅ {dataset_name} KAN+Attention Complete!")
    print(f"   Optimal Threshold: {optimal_threshold:.2f}")
    print(f"   Test F2: {test_metrics['f2']:.4f}")
    print(f"   Test Recall: {test_metrics['recall']:.4f} {'🎯' if test_metrics['recall'] >= 0.80 else '⚠️'}")
    print(f"   Test Accuracy: {test_metrics['accuracy']:.4f}")
    
    # Compare with baseline
    baseline_f2 = baseline_results[dataset_name]['test_metrics']['f2']
    kan_base_f2 = kan_base_results[dataset_name]['test_metrics']['f2']
    improvement_vs_baseline = (test_metrics['f2'] - baseline_f2) / baseline_f2 * 100
    improvement_vs_kan = (test_metrics['f2'] - kan_base_f2) / kan_base_f2 * 100
    
    print(f"\n   📈 Improvement:")
    print(f"      vs Baseline RF: {improvement_vs_baseline:+.1f}%")
    print(f"      vs KAN Base:    {improvement_vs_kan:+.1f}%")

print(f"\n{'='*70}")
print(f"✅ ALL KAN+ATTENTION MODELS TRAINED")
print(f"{'='*70}")

---
## 🌟 EXECUTION: KAN + Feature Attention (ÖZGÜN KATKI)

Feature-level attention eklendi - Her sample için feature'lara ağırlık

In [ ]:
# ============================================================================
# KAN BASE - LIGHTWEIGHT CONFIG (CPU OPTIMIZED)
# ============================================================================

print("\n" + "="*70)
print("🔥 KAN BASE MODELS - LIGHTWEIGHT (Focal Loss)")
print("="*70)

# Lightweight config for CPU
KAN_LITE_CONFIG = {
    'hidden_dim': 32,      # Small hidden layer
    'grid_size': 3,        # Small grid
    'spline_order': 2,     # Low spline order
    'learning_rate': 0.01,
    'epochs': 50,          # Fewer epochs
    'batch_size': 64,      # Larger batches (faster)
    'patience': 10,
    'loss_type': 'Focal'
}

print(f"\n⚡ Lightweight Config:")
print(f"   Hidden: {KAN_LITE_CONFIG['hidden_dim']}, Grid: {KAN_LITE_CONFIG['grid_size']}, Spline: {KAN_LITE_CONFIG['spline_order']}")
print(f"   Epochs: {KAN_LITE_CONFIG['epochs']}, Batch: {KAN_LITE_CONFIG['batch_size']}")
print(f"   Loss: {KAN_LITE_CONFIG['loss_type']}\n")

kan_base_results = {}

for dataset_name, data in prepared_datasets.items():
    print(f"\n{'='*70}")
    print(f"📊 Dataset: {dataset_name}")
    print(f"{'='*70}")
    
    # Create lightweight KAN model
    model = KAN(
        input_dim=data['n_features'],
        hidden_dim=KAN_LITE_CONFIG['hidden_dim'],
        grid_size=KAN_LITE_CONFIG['grid_size'],
        spline_order=KAN_LITE_CONFIG['spline_order'],
        dropout=0.3
    )
    
    # Train
    model, history = train_kan_model(
        model,
        data['X_train_smote'],
        data['y_train_smote'],
        data['X_val'],
        data['y_val'],
        learning_rate=KAN_LITE_CONFIG['learning_rate'],
        epochs=KAN_LITE_CONFIG['epochs'],
        batch_size=KAN_LITE_CONFIG['batch_size'],
        loss_type=KAN_LITE_CONFIG['loss_type'],
        patience=KAN_LITE_CONFIG['patience'],
        verbose=True
    )
    
    # Evaluate on validation
    model.eval()
    with torch.no_grad():
        X_val_t = torch.FloatTensor(data['X_val']).to(device)
        y_val_proba = model(X_val_t).cpu().numpy().flatten()
    
    # Find optimal threshold
    print(f"\n🔍 Finding optimal threshold on validation set...")
    threshold_result = find_optimal_threshold(
        data['y_val'],
        y_val_proba,
        metric='f2',
        threshold_range=CONFIG['threshold_range'],
        step=CONFIG['threshold_step'],
        accuracy_floor=CONFIG['gwo_accuracy_floor'],
        verbose=True
    )
    
    optimal_threshold = threshold_result['threshold']
    
    # Evaluate on test
    print(f"\n🧪 Evaluating on TEST set with threshold={optimal_threshold:.2f}...")
    with torch.no_grad():
        X_test_t = torch.FloatTensor(data['X_test']).to(device)
        y_test_proba = model(X_test_t).cpu().numpy().flatten()
    
    y_test_pred = (y_test_proba >= optimal_threshold).astype(int)
    test_metrics = calculate_all_metrics(data['y_test'], y_test_pred, y_test_proba)
    
    print(f"\n   📊 Test Performance:")
    print_metrics(test_metrics, prefix="   ")
    
    # Store results
    kan_base_results[dataset_name] = {
        'model': model,
        'optimal_threshold': optimal_threshold,
        'val_metrics': threshold_result['metrics'],
        'test_metrics': test_metrics,
        'history': history
    }
    
    print(f"\n✅ {dataset_name} KAN Base Complete!")
    print(f"   Optimal Threshold: {optimal_threshold:.2f}")
    print(f"   Test F2: {test_metrics['f2']:.4f}")
    print(f"   Test Recall: {test_metrics['recall']:.4f} {'🎯' if test_metrics['recall'] >= 0.80 else '⚠️'}")
    print(f"   Test Accuracy: {test_metrics['accuracy']:.4f}")

print(f"\n{'='*70}")
print(f"✅ ALL KAN BASE MODELS TRAINED")
print(f"{'='*70}")

# NASA Defect Prediction: Comprehensive GWO-KAN Pipeline
## F2 & Recall Optimized with Feature-Level Attention

**Hedef:** F2 ve Recall (defective=1) maksimizasyonu, Accuracy korunarak

**Datasets:** JM1, KC1

**Kısıtlar:**
- ✅ Test seti leakage YOK (scaler fit sadece train)
- ✅ SMOTE sadece train seti
- ✅ Threshold tuning val seti (F2 hedef)
- ✅ Colab CPU optimize

**Özgün Katkı:** Feature-Level Attention Mechanism

---
## 1. Config / Seed / Run Kayıt
Her deneyin izlenebilirliği için config ve seed yönetimi

In [ ]:
# ============================================================================
# GOOGLE DRIVE MOUNT (REQUIRED FOR COLAB)
# ============================================================================

print("="*70)
print("📂 MOUNTING GOOGLE DRIVE")
print("="*70)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("\n✅ Google Drive mounted successfully!")
    print(f"   Dataset path: /content/drive/MyDrive/nasa-defect-gwo-kan/dataset")
except ImportError:
    print("\n⚠️  Not running on Colab - skipping Drive mount")
except Exception as e:
    print(f"\n❌ Drive mount failed: {e}")
    print(f"   Please mount manually or check permissions")

print("="*70)

In [ ]:
import os
import json
import warnings
import datetime
from pathlib import Path
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

# ============================================================================
# EXPERIMENT CONFIG
# ============================================================================

CONFIG = {
    # Run Info
    'run_id': datetime.datetime.now().strftime('%Y%m%d_%H%M%S'),
    'experiment_name': 'NASA_Defect_F2_Optimized',
    
    # Random Seeds
    'seed': 42,
    
    # Dataset Config
    'dataset_path': '/content/drive/MyDrive/nasa-defect-gwo-kan/dataset',
    'datasets': ['JM1', 'KC1'],  # Focus datasets
    
    # Split Config
    'test_size': 0.2,
    'val_size': 0.2,  # from train
    
    # SMOTE Config
    'smote_ratio': 0.7,  # Primary ratio
    'smote_alternatives': [0.5, 0.9],  # For ablation
    
    # Threshold Config
    'threshold_range': (0.05, 0.95),
    'threshold_step': 0.05,
    'threshold_metric': 'F2',  # Optimize for F2
    
    # Model Config
    'baseline_model': 'RandomForest',
    
    # KAN Config
    'kan_grid_size': [3, 5, 7],
    'kan_spline_order': [2, 3, 4],
    'kan_hidden_dim': [32, 64, 128],
    'kan_learning_rate': [0.001, 0.01, 0.05],
    'kan_epochs': 100,
    'kan_batch_size': 32,
    'kan_patience': 15,
    
    # GWO Config
    'gwo_n_wolves': 10,
    'gwo_n_iterations': 20,
    'gwo_fitness_weights': {
        'f1': 0.5,
        'recall': 0.3,
        'accuracy': 0.2
    },
    'gwo_accuracy_floor': 0.5,  # Minimum acceptable accuracy
    
    # Loss Config
    'loss_types': ['WeightedBCE', 'Focal'],
    'focal_alpha': 0.25,
    'focal_gamma': 2.0,
    
    # Attention Config
    'attention_dim': 16,
    'attention_dropout': 0.2,
    
    # Output Config
    'save_models': True,
    'output_dir': './results',
    'export_format': ['csv', 'json', 'xlsx']
}

# Set all random seeds
def set_seed(seed):
    np.random.seed(seed)
    import random
    random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except ImportError:
        pass

set_seed(CONFIG['seed'])

# Create output directory
os.makedirs(CONFIG['output_dir'], exist_ok=True)

# Save config
config_path = os.path.join(CONFIG['output_dir'], f"config_{CONFIG['run_id']}.json")
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("="*70)
print(f"🔧 EXPERIMENT CONFIG INITIALIZED")
print("="*70)
print(f"Run ID: {CONFIG['run_id']}")
print(f"Experiment: {CONFIG['experiment_name']}")
print(f"Random Seed: {CONFIG['seed']}")
print(f"Datasets: {', '.join(CONFIG['datasets'])}")
print(f"Config saved to: {config_path}")
print("="*70)

---
## 2. Dependencies & Imports

In [ ]:
# ============================================================================
# INSTALL & IMPORT DEPENDENCIES
# ============================================================================

# Install required packages (if on Colab)
try:
    import google.colab
    IN_COLAB = True
    print("📍 Running on Google Colab")
    !pip install imbalanced-learn scipy scikit-learn torch matplotlib seaborn pandas numpy openpyxl -q
except ImportError:
    IN_COLAB = False
    print("📍 Running locally")

# Core imports
import glob
from io import StringIO
from collections import defaultdict

# Scientific computing
import numpy as np
import pandas as pd
from scipy.io import arff

# Machine learning
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    fbeta_score, roc_auc_score, balanced_accuracy_score,
    confusion_matrix, classification_report, average_precision_score
)
from imblearn.over_sampling import SMOTE

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ All dependencies loaded successfully!")
print(f"🖥️  Device: {device}")
print(f"🐍 PyTorch version: {torch.__version__}")

---
## 3. Dataset Keşfi ve Özet Tablo

JM1 ve KC1 datasetlerinin temel istatistikleri

In [ ]:
# ============================================================================
# DATASET LOADING UTILITIES
# ============================================================================

def load_arff_dataset(file_path):
    """
    Load ARFF file with robust error handling
    
    Parameters:
    -----------
    file_path : str
        Path to ARFF file
        
    Returns:
    --------
    pd.DataFrame
        Loaded dataset
    """
    try:
        data, meta = arff.loadarff(file_path)
        df = pd.DataFrame(data)
        
        # Decode byte strings
        for col in df.columns:
            if df[col].dtype == object:
                try:
                    df[col] = df[col].str.decode('utf-8')
                except AttributeError:
                    pass
        
        return df
    
    except Exception as e:
        print(f"⚠️  scipy.io.arff failed: {e}\n" + f"🔄 Trying alternative parsing...")
        
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
        
        data_start = content.lower().find('@data')
        if data_start == -1:
            raise ValueError("No @data section found")
        
        data_section = content[data_start + 5:].strip()
        df = pd.read_csv(StringIO(data_section), header=None)
        
        return df


def get_dataset_summary(df, dataset_name):
    """
    Generate summary statistics for a dataset
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset
    dataset_name : str
        Name of dataset
        
    Returns:
    --------
    dict
        Summary statistics
    """
    # Separate features and labels
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    # Encode labels if needed
    if y.dtype == object:
        le = LabelEncoder()
        y = le.fit_transform(y)
    else:
        y = y.astype(int)
    
    # Calculate statistics
    n_samples = len(y)
    n_features = X.shape[1]
    n_defective = np.sum(y == 1)
    n_clean = np.sum(y == 0)
    defect_ratio = n_defective / n_samples
    imbalance_ratio = n_clean / n_defective if n_defective > 0 else 0
    
    return {
        'Dataset': dataset_name,
        'n_samples': n_samples,
        'n_features': n_features,
        'n_defective': n_defective,
        'n_clean': n_clean,
        'defect_ratio': defect_ratio,
        'imbalance_ratio': imbalance_ratio
    }


# ============================================================================
# LOAD AND EXPLORE DATASETS
# ============================================================================

print("\n" + "="*70)
print("📊 DATASET EXPLORATION - JM1 & KC1")
print("="*70)

# Load datasets
dataset_summaries = []
raw_datasets = {}

for dataset_name in CONFIG['datasets']:
    file_path = os.path.join(CONFIG['dataset_path'], f"{dataset_name}.arff")
    
    print(f"\n📁 Loading {dataset_name}...")
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        continue
    
    try:
        df = load_arff_dataset(file_path)
        raw_datasets[dataset_name] = df
        
        summary = get_dataset_summary(df, dataset_name)
        dataset_summaries.append(summary)
        
        print(f"✅ Loaded successfully: {summary['n_samples']} samples, {summary['n_features']} features\n" + f"   Defective: {summary['n_defective']} ({summary['defect_ratio']:.2%})")
        print(f"   Clean: {summary['n_clean']} ({1-summary['defect_ratio']:.2%})\n" + f"   Imbalance Ratio: {summary['imbalance_ratio']:.2f}:1")
        
    except Exception as e:
        print(f"❌ Failed to load {dataset_name}: {e}")
        import traceback
        traceback.print_exc()

# Create summary table
summary_df = pd.DataFrame(dataset_summaries)

print("\n" + "="*70)
print("📋 DATASET SUMMARY TABLE")
print("="*70)
print(summary_df.to_string(index=False))

# Save summary
summary_path = os.path.join(CONFIG['output_dir'], f"dataset_summary_{CONFIG['run_id']}.csv")
summary_df.to_csv(summary_path, index=False)
print(f"\n💾 Summary saved to: {summary_path}")

---
## 4. Split + Scaling (Leakage Prevention)

**KRITIK:** Scaler sadece train'e fit, val/test'e transform

In [ ]:
# ============================================================================
# LEAKAGE-FREE DATA PREPARATION
# ============================================================================

def prepare_dataset_splits(df, dataset_name, config):
    """
    Prepare train/val/test splits with NO LEAKAGE
    
    Pipeline:
    1. Split into train/test (stratified)
    2. Split train into train/val (stratified)
    3. Fit scaler ONLY on train
    4. Transform train/val/test with fitted scaler
    5. Apply SMOTE ONLY on train
    
    Parameters:
    -----------
    df : pd.DataFrame
        Raw dataset
    dataset_name : str
        Name of dataset
    config : dict
        Configuration dictionary
        
    Returns:
    --------
    dict
        Dictionary with train/val/test splits and metadata
    """
    print(f"\n{'='*70}\n" + f"🔧 PREPARING {dataset_name} - LEAKAGE-FREE PIPELINE")
    print(f"{'='*70}")
    
    # Step 1: Extract features and labels
    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values
    
    # Encode labels
    if y.dtype == object:
        le = LabelEncoder()
        y = le.fit_transform(y)
    else:
        y = y.astype(int)
    
    # Handle missing values
    if np.any(np.isnan(X)):
        print("⚠️  Handling missing values with median imputation")
        col_median = np.nanmedian(X, axis=0)
        inds = np.where(np.isnan(X))
        X[inds] = np.take(col_median, inds[1])
    
    print(f"\n📊 Original Data:\n" + f"   Total Samples: {len(y)}")
    print(f"   Features: {X.shape[1]}\n" + f"   Defective: {np.sum(y==1)} ({np.mean(y==1):.2%})")
    print(f"   Clean: {np.sum(y==0)} ({np.mean(y==0):.2%})")
    
    # Step 2: Train/Test Split (stratified)
    print(f"\n🔀 Step 1: Train/Test Split (test_size={config['test_size']})")
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y,
        test_size=config['test_size'],
        stratify=y,
        random_state=config['seed']
    )
    
    print(f"   Train: {len(y_train_full)} samples (defect: {np.mean(y_train_full==1):.2%})\n" + f"   Test:  {len(y_test)} samples (defect: {np.mean(y_test==1):.2%})")
    
    # Step 3: Train/Val Split (from train, stratified)
    print(f"\n🔀 Step 2: Train/Val Split (val_size={config['val_size']})")
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full,
        test_size=config['val_size'],
        stratify=y_train_full,
        random_state=config['seed']
    )
    
    print(f"   Train: {len(y_train)} samples (defect: {np.mean(y_train==1):.2%})\n" + f"   Val:   {len(y_val)} samples (defect: {np.mean(y_val==1):.2%})")
    print(f"   Test:  {len(y_test)} samples (defect: {np.mean(y_test==1):.2%})")
    
    # Step 4: Scaling (FIT ONLY ON TRAIN)
    print(f"\n📏 Step 3: Feature Scaling (MinMaxScaler)\n" + f"   ⚠️  CRITICAL: Scaler FIT only on train set")
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)  # Only transform
    X_test_scaled = scaler.transform(X_test)  # Only transform
    
    print(f"   ✅ Train: fitted and transformed\n" + f"   ✅ Val: transformed (no fit)")
    print(f"   ✅ Test: transformed (no fit)")
    
    # Verify no leakage
    print(f"\n🔍 Leakage Check:\n" + f"   Train min/max: [{X_train_scaled.min():.4f}, {X_train_scaled.max():.4f}]")
    print(f"   Val min/max:   [{X_val_scaled.min():.4f}, {X_val_scaled.max():.4f}]\n" + f"   Test min/max:  [{X_test_scaled.min():.4f}, {X_test_scaled.max():.4f}]")
    
    if X_val_scaled.min() < -0.01 or X_val_scaled.max() > 1.01:
        print(f"   ⚠️  Val data outside [0,1] range - expected behavior (no leakage)")
    if X_test_scaled.min() < -0.01 or X_test_scaled.max() > 1.01:
        print(f"   ⚠️  Test data outside [0,1] range - expected behavior (no leakage)")
    
    return {
        'dataset_name': dataset_name,
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_val': y_val,
        'y_test': y_test,
        'scaler': scaler,
        'n_features': X.shape[1]
    }


# ============================================================================
# PREPARE ALL DATASETS
# ============================================================================

prepared_datasets = {}

for dataset_name in CONFIG['datasets']:
    if dataset_name in raw_datasets:
        prepared_datasets[dataset_name] = prepare_dataset_splits(
            raw_datasets[dataset_name],
            dataset_name,
            CONFIG
        )
    else:
        print(f"\n❌ Skipping {dataset_name} - not loaded\n" + f"\n{'='*70}")
print(f"✅ ALL DATASETS PREPARED - NO LEAKAGE")
print(f"{'='*70}")

---
## 5. Imbalance Stratejisi - SMOTE

SMOTE 0.7 (primary) ve alternatif oranlar (0.5, 0.9)

In [ ]:
# ============================================================================
# SMOTE STRATEGY - TRAIN ONLY
# ============================================================================

def apply_smote(X_train, y_train, ratio, seed):
    """
    Apply SMOTE to training data ONLY
    
    Parameters:
    -----------
    X_train : np.ndarray
        Training features
    y_train : np.ndarray
        Training labels
    ratio : float
        SMOTE sampling ratio (minority will be ratio * majority)
    seed : int
        Random seed
        
    Returns:
    --------
    tuple
        (X_train_smote, y_train_smote)
    """
    print(f"\n🔄 Applying SMOTE (ratio={ratio})...\n" + f"   Before SMOTE:")
    print(f"     Total: {len(y_train)}\n" + f"     Clean: {np.sum(y_train==0)} ({np.mean(y_train==0):.2%})")
    print(f"     Defective: {np.sum(y_train==1)} ({np.mean(y_train==1):.2%})")
    
    try:
        smote = SMOTE(sampling_strategy=ratio, random_state=seed)
        X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
        
        print(f"   After SMOTE:\n" + f"     Total: {len(y_train_smote)} (+{len(y_train_smote)-len(y_train)})")
        print(f"     Clean: {np.sum(y_train_smote==0)} ({np.mean(y_train_smote==0):.2%})\n" + f"     Defective: {np.sum(y_train_smote==1)} ({np.mean(y_train_smote==1):.2%})")
        print(f"   ✅ SMOTE successful")
        
        return X_train_smote, y_train_smote
    
    except Exception as e:
        print(f"   ❌ SMOTE failed: {e}\n" + f"   ⚠️  Continuing without SMOTE")
        return X_train, y_train


# ============================================================================
# APPLY SMOTE TO ALL DATASETS
# ============================================================================

print("\n" + "="*70)
print("🎯 IMBALANCE HANDLING - SMOTE (Train Only)")
print("="*70)

# Apply primary SMOTE ratio (0.7)
for dataset_name, data in prepared_datasets.items():
    print(f"\n📊 Dataset: {dataset_name}")
    
    X_train_smote, y_train_smote = apply_smote(
        data['X_train'],
        data['y_train'],
        CONFIG['smote_ratio'],
        CONFIG['seed']
    )
    
    # Store SMOTE-augmented training data
    data['X_train_smote'] = X_train_smote
    data['y_train_smote'] = y_train_smote
    
    # CRITICAL: Val and Test remain UNCHANGED
    print(f"\n   🔒 Val and Test sets: UNCHANGED (no SMOTE)\n" + f"      Val: {len(data['y_val'])} samples")
    print(f"      Test: {len(data['y_test'])} samples\n" + f"\n{'='*70}")
print(f"✅ SMOTE APPLIED - VAL/TEST UNTOUCHED")
print(f"{'='*70}")

---
## 6. Metrics - Unified Function

Tek fonksiyon, tüm metrikler (Recall, Precision, F1, F2, Accuracy, Balanced Accuracy, PR-AUC)

In [ ]:
# ============================================================================
# UNIFIED METRICS CALCULATION
# ============================================================================

def calculate_all_metrics(y_true, y_pred, y_pred_proba=None):
    """
    Calculate all required metrics in one function
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    y_pred_proba : array-like, optional
        Predicted probabilities (for AUC metrics)
        
    Returns:
    --------
    dict
        Dictionary with all metrics
    """
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Basic metrics
    metrics = {
        # Classification metrics
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'f2': fbeta_score(y_true, y_pred, beta=2, zero_division=0),
        'accuracy': accuracy_score(y_true, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        
        # Confusion matrix components
        'tp': int(tp),
        'fp': int(fp),
        'tn': int(tn),
        'fn': int(fn)
    }
    
    # Add AUC metrics if probabilities provided
    if y_pred_proba is not None:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba)
        except:
            metrics['roc_auc'] = 0.0
        
        try:
            metrics['pr_auc'] = average_precision_score(y_true, y_pred_proba)
        except:
            metrics['pr_auc'] = 0.0
    else:
        metrics['roc_auc'] = 0.0
        metrics['pr_auc'] = 0.0
    
    return metrics


def print_metrics(metrics, prefix=""):
    """
    Pretty print metrics
    
    Parameters:
    -----------
    metrics : dict
        Metrics dictionary
    prefix : str
        Prefix for print statements
    """
    print(f"{prefix}📊 Performance Metrics:\n" + f"{prefix}   Recall (Defective):    {metrics['recall']:.4f} ⭐")
    print(f"{prefix}   Precision (Defective): {metrics['precision']:.4f}\n" + f"{prefix}   F1-Score:              {metrics['f1']:.4f}")
    print(f"{prefix}   F2-Score:              {metrics['f2']:.4f} 🎯\n" + f"{prefix}   Accuracy:              {metrics['accuracy']:.4f}")
    print(f"{prefix}   Balanced Accuracy:     {metrics['balanced_accuracy']:.4f}\n" + f"{prefix}   PR-AUC:                {metrics['pr_auc']:.4f}")
    print(f"{prefix}   ROC-AUC:               {metrics['roc_auc']:.4f}\n" + f"{prefix}\n   Confusion Matrix:")
    print(f"{prefix}      TP: {metrics['tp']:4d}  FP: {metrics['fp']:4d}\n" + f"{prefix}      FN: {metrics['fn']:4d}  TN: {metrics['tn']:4d}")


# Test metrics function
print("\n" + "="*70)
print("🧪 METRICS FUNCTION TEST")
print("="*70)

# Create dummy data for testing
y_true_test = np.array([0, 1, 1, 0, 1, 0, 1, 1, 0, 1])
y_pred_test = np.array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1])
y_proba_test = np.array([0.2, 0.8, 0.9, 0.1, 0.4, 0.3, 0.7, 0.85, 0.6, 0.75])

test_metrics = calculate_all_metrics(y_true_test, y_pred_test, y_proba_test)
print_metrics(test_metrics)

print("\n✅ Metrics function ready!")

---
## 7. Threshold Tuning

Val seti üzerinde F2 maksimizasyonu için threshold optimizasyonu

In [ ]:
# ============================================================================
# THRESHOLD OPTIMIZATION (F2-FOCUSED)
# ============================================================================

def find_optimal_threshold(y_true, y_pred_proba, metric='f2', 
                          threshold_range=(0.05, 0.95), step=0.05,
                          accuracy_floor=0.5, verbose=True):
    """
    Find optimal classification threshold by maximizing target metric
    with accuracy guardrail
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred_proba : array-like
        Predicted probabilities
    metric : str
        Metric to optimize ('f1', 'f2', 'recall')
    threshold_range : tuple
        (min_threshold, max_threshold)
    step : float
        Step size for threshold search
    accuracy_floor : float
        Minimum acceptable accuracy
    verbose : bool
        Print search results
        
    Returns:
    --------
    dict
        {'threshold': best_threshold, 'metrics': best_metrics, 'curve': threshold_curve}
    """
    thresholds = np.arange(threshold_range[0], threshold_range[1] + step, step)
    
    results = []
    best_score = -1
    best_threshold = 0.5
    best_metrics = None
    
    if verbose:
        print(f"\n🔍 Threshold Optimization (Target: {metric.upper()})\n" + f"   Range: [{threshold_range[0]}, {threshold_range[1]}]")
        print(f"   Step: {step}\n" + f"   Accuracy Floor: {accuracy_floor:.2f}\n")
    
    for thresh in thresholds:
        y_pred = (y_pred_proba >= thresh).astype(int)
        metrics = calculate_all_metrics(y_true, y_pred, y_pred_proba)
        
        # Get target metric score
        if metric.lower() == 'f2':
            score = metrics['f2']
        elif metric.lower() == 'f1':
            score = metrics['f1']
        elif metric.lower() == 'recall':
            score = metrics['recall']
        else:
            score = metrics['f2']  # default
        
        # Apply accuracy guardrail
        if metrics['accuracy'] < accuracy_floor:
            score = 0  # Penalize thresholds with too low accuracy
        
        results.append({
            'threshold': thresh,
            'score': score,
            **metrics
        })
        
        # Track best
        if score > best_score:
            best_score = score
            best_threshold = thresh
            best_metrics = metrics.copy()
    
    results_df = pd.DataFrame(results)
    
    if verbose:
        print(f"\n✅ Optimal Threshold: {best_threshold:.2f}\n" + f"   {metric.upper()}: {best_score:.4f}")
        print(f"   Accuracy: {best_metrics['accuracy']:.4f}\n" + f"   Recall: {best_metrics['recall']:.4f}")
        print(f"   Precision: {best_metrics['precision']:.4f}")
        
        # Show top 5 thresholds
        print(f"\n   Top 5 Thresholds:")
        top5 = results_df.nlargest(5, 'score')[['threshold', 'score', 'recall', 'precision', 'f2', 'accuracy']]
        for idx, row in top5.iterrows():
            print(f"      {row['threshold']:.2f} → {metric.upper()}={row['score']:.4f} "
                  f"(Rec={row['recall']:.3f}, Prec={row['precision']:.3f}, Acc={row['accuracy']:.3f})")
    
    return {
        'threshold': best_threshold,
        'metrics': best_metrics,
        'curve': results_df
    }


# Test threshold function
print("\n" + "="*70)
print("🧪 THRESHOLD TUNING TEST")
print("="*70)

# Use dummy data
y_true_dummy = np.random.binomial(1, 0.3, 100)
y_proba_dummy = np.random.beta(2, 5, 100)

threshold_result = find_optimal_threshold(
    y_true_dummy, 
    y_proba_dummy,
    metric='f2',
    threshold_range=(0.1, 0.9),
    step=0.1,
    accuracy_floor=0.4,
    verbose=True
)

print("\n✅ Threshold tuning function ready!")

---
## 8. Baseline Model - Random Forest

Class-weighted Random Forest as baseline

---
## 📋 NOTEBOOK İÇERİK ÖZETİ

Bu notebook aşağıdaki bölümleri içerir:

### ✅ Tamamlanan Bölümler:

1. **Config & Seed Management** - İzlenebilirlik için run tracking
2. **Dataset Exploration** - JM1 & KC1 özet istatistikler
3. **Leakage-Free Split** - Train/Val/Test ayrımı (scaler leakage yok)
4. **SMOTE Strategy** - 0.7 ratio (sadece train'e uygulanır)
5. **Unified Metrics** - Recall, Precision, F1, F2, Accuracy, Balanced Acc, PR-AUC
6. **Threshold Tuning** - Val seti üzerinde F2 maksimizasyonu
7. **Baseline RF** - Class-weighted Random Forest
8. **KAN Implementation** - Spline-based activation functions
9. **Loss Functions** - Weighted BCE & Focal Loss
10. **GWO Optimizer** - Multi-metric fitness (0.5\*F1 + 0.3\*Recall + 0.2\*Acc)
11. **Feature Attention** - Özgün katkı: Sample-specific feature weighting
12. **Export Functions** - CSV, JSON, XLSX formatında sonuç export

### 🎯 Ana Hedefler:

- ✅ **F2 ve Recall maksimizasyonu** (defective=1 için)
- ✅ **Accuracy korunması** (GWO accuracy floor ile)
- ✅ **Leakage prevention** (test seti tamamen izole)
- ✅ **Colab CPU optimize** (GPU gereksiz)

### 🚀 Özgün Katkı:

**Feature-Level Attention Mechanism**
- Lightweight attention layer (transformer değil)
- Her sample için feature'lara dinamik ağırlık
- KAN'a entegre edilmiş
- Interpretable (attention weights incelenebilir)

### 📊 Beklenen Çıktılar:

1. `dataset_summary_<run_id>.csv` - Dataset istatistikleri
2. `config_<run_id>.json` - Experiment config
3. `final_results_<run_id>.csv/json/xlsx` - Tüm model sonuçları
4. `comparison_plot_<run_id>.png` - Karşılaştırma grafiği

### 🔬 Experimental Protocol:

**Metrikler (öncelik sırasına göre):**
1. F2-Score (defect detection için en önemli)
2. Recall (safety-critical)
3. Precision (false positive kontrolü)
4. Accuracy (genel performans)
5. Balanced Accuracy (imbalanced data için)
6. PR-AUC (overall ranking)

**Threshold Tuning:**
- Val seti üzerinde 0.05-0.95 arası grid search
- F2 maksimizasyonu hedefi
- Accuracy floor guardrail (0.5)

**GWO Fitness:**
```
fitness = 0.5 * F1 + 0.3 * Recall + 0.2 * Accuracy
if accuracy < 0.5:
    fitness *= 0.1  # Heavy penalty
```

---

**👨‍💻 Hazırlayan:** Claude (Anthropic)  
**📅 Tarih:** 2026-01-05  
**🎯 Proje:** NASA Defect Prediction with GWO-KAN  
**📚 Datasets:** JM1, KC1  

---

---
## 13. EXECUTION TEMPLATE

Aşağıdaki hücreleri sırayla çalıştırarak tüm pipeline'ı çalıştırabilirsiniz:

1. **Baseline RF**: Her dataset için Random Forest baseline
2. **KAN Base**: Temel KAN modeli (Focal Loss ile)
3. **KAN + GWO**: GWO optimized hyperparameters
4. **KAN + Attention**: Feature-level attention ile KAN
5. **Final GWO + Attention**: GWO optimized + Attention (En iyi model)
6. **Loss Ablation**: WeightedBCE vs Focal karşılaştırması
7. **Compile & Export**: Tüm sonuçları derle ve export et

### Örnek Çalıştırma:

```python
# Example: Train baseline for one dataset
dataset_name = 'JM1'
data = prepared_datasets[dataset_name]

# Train RF baseline
rf = train_baseline_rf(
    data['X_train_smote'],
    data['y_train_smote'],
    data['X_val'],
    data['y_val'],
    CONFIG['seed']
)

# Evaluate with threshold tuning
results = evaluate_baseline_with_threshold(
    rf,
    data['X_val'],
    data['y_val'],
    data['X_test'],
    data['y_test'],
    CONFIG
)

print(f\"Baseline F2: {results['test_metrics']['f2']:.4f}\")
```

**NOT:** Notebook'u Colab'da çalıştırırken Google Drive'ı mount etmeyi unutmayın:

```python
from google.colab import drive
drive.mount('/content/drive')
```

In [ ]:
# ============================================================================
# FINAL RESULTS COMPILATION AND EXPORT
# ============================================================================

def compile_final_results(all_results, config):
    """
    Compile all experimental results into a summary table
    
    Parameters:
    -----------
    all_results : dict
        Dictionary with all experimental results
    config : dict
        Experiment configuration
        
    Returns:
    --------
    pd.DataFrame
        Final summary table
    """
    final_rows = []
    
    for dataset_name, dataset_results in all_results.items():
        # Get dataset info
        dataset_info = dataset_results.get('dataset_info', {})
        
        # Compile each model's results
        for model_name, model_results in dataset_results.items():
            if model_name == 'dataset_info':
                continue
            
            if 'test_metrics' not in model_results:
                continue
            
            metrics = model_results['test_metrics']
            
            row = {
                'dataset': dataset_name,
                'model': model_name,
                'defect_ratio': dataset_info.get('defect_ratio', 0),
                'n_samples': dataset_info.get('n_samples', 0),
                'n_features': dataset_info.get('n_features', 0),
                'recall': metrics['recall'],
                'precision': metrics['precision'],
                'f1': metrics['f1'],
                'f2': metrics['f2'],
                'accuracy': metrics['accuracy'],
                'balanced_acc': metrics['balanced_accuracy'],
                'pr_auc': metrics['pr_auc'],
                'threshold': model_results.get('optimal_threshold', 0.5)
            }
            
            final_rows.append(row)
    
    df = pd.DataFrame(final_rows)
    
    return df


def export_results(results_df, config):
    """
    Export results to multiple formats
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results dataframe
    config : dict
        Configuration
    """
    base_path = os.path.join(config['output_dir'], f"final_results_{config['run_id']}")
    
    # CSV
    if 'csv' in config['export_format']:
        csv_path = f"{base_path}.csv"
        results_df.to_csv(csv_path, index=False)
        print(f"✅ CSV saved: {csv_path}")
    
    # JSON
    if 'json' in config['export_format']:
        json_path = f"{base_path}.json"
        results_df.to_json(json_path, orient='records', indent=2)
        print(f"✅ JSON saved: {json_path}")
    
    # Excel
    if 'xlsx' in config['export_format']:
        xlsx_path = f"{base_path}.xlsx"
        try:
            results_df.to_excel(xlsx_path, index=False, sheet_name='Results')
            print(f"✅ Excel saved: {xlsx_path}")
        except Exception as e:
            print(f"⚠️  Excel export failed: {e}")


def print_final_summary(results_df):
    """
    Print formatted final summary
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results dataframe
    """
    print("\n" + "="*80)
    print(" FINAL RESULTS SUMMARY ".center(80, "="))
    print("="*80)
    
    # Per-dataset results
    for dataset in results_df['dataset'].unique():
        dataset_df = results_df[results_df['dataset'] == dataset]
        
        print(f"\n📊 Dataset: {dataset}")
        print("-" * 80)
        
        for _, row in dataset_df.iterrows():
            print(f"\n   {row['model']}:")
            print(f"      Recall:     {row['recall']:.4f} ⭐")
            print(f"      Precision:  {row['precision']:.4f}")
            print(f"      F1:         {row['f1']:.4f}")
            print(f"      F2:         {row['f2']:.4f} 🎯")
            print(f"      Accuracy:   {row['accuracy']:.4f}")
            print(f"      PR-AUC:     {row['pr_auc']:.4f}")
            print(f"      Threshold:  {row['threshold']:.3f}")
    
    # Average across datasets
    print("\n" + "="*80)
    print(" AVERAGE ACROSS DATASETS ".center(80, "="))
    print("="*80)
    
    for model in results_df['model'].unique():
        model_df = results_df[results_df['model'] == model]
        
        print(f"\n   {model}:")
        print(f"      Recall:     {model_df['recall'].mean():.4f} ⭐")
        print(f"      Precision:  {model_df['precision'].mean():.4f}")
        print(f"      F1:         {model_df['f1'].mean():.4f}")
        print(f"      F2:         {model_df['f2'].mean():.4f} 🎯")
        print(f"      Accuracy:   {model_df['accuracy'].mean():.4f}")
        print(f"      PR-AUC:     {model_df['pr_auc'].mean():.4f}")
    
    print("\n" + "="*80)


def create_comparison_plot(results_df, config):
    """
    Create comparison visualization
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results dataframe
    config : dict
        Configuration
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Model Performance Comparison - JM1 & KC1', 
                 fontsize=16, fontweight='bold')
    
    metrics = ['recall', 'precision', 'f1', 'f2', 'accuracy', 'pr_auc']
    metric_names = ['Recall', 'Precision', 'F1-Score', 'F2-Score', 'Accuracy', 'PR-AUC']
    
    for idx, (metric, name) in enumerate(zip(metrics, metric_names)):
        ax = axes[idx // 3, idx % 3]
        
        # Group by dataset and model
        pivot = results_df.pivot(index='model', columns='dataset', values=metric)
        
        pivot.plot(kind='bar', ax=ax, width=0.8)
        ax.set_title(name, fontsize=12, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel(name)
        ax.set_ylim(0, 1.0)
        ax.legend(title='Dataset', loc='lower right')
        ax.grid(axis='y', alpha=0.3)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    
    # Save plot
    plot_path = os.path.join(config['output_dir'], 
                            f"comparison_plot_{config['run_id']}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"\n✅ Comparison plot saved: {plot_path}")
    
    plt.show()


print("\n✅ Export functions ready!")

---
## 12. Final Tables & Export

Tüm sonuçları toplama ve raporlama

In [ ]:
# ============================================================================
# FEATURE-LEVEL ATTENTION MECHANISM (ÖZGÜN KATKI)
# ============================================================================

class FeatureAttention(nn.Module):
    """
    Lightweight Feature-Level Attention
    
    Learns to weight input features based on their importance
    for each sample (sample-specific attention).
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    attention_dim : int
        Attention hidden dimension
    dropout : float
        Dropout rate
    """
    
    def __init__(self, input_dim, attention_dim=16, dropout=0.2):
        super(FeatureAttention, self).__init__()
        
        self.input_dim = input_dim
        self.attention_dim = attention_dim
        
        # Attention network (small MLP)
        self.attention_fc1 = nn.Linear(input_dim, attention_dim)
        self.attention_fc2 = nn.Linear(attention_dim, input_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """
        Forward pass - compute attention weights and apply them
        
        Parameters:
        -----------
        x : torch.Tensor
            Input features (batch_size, input_dim)
            
        Returns:
        --------
        tuple
            (attended_features, attention_weights)
        """
        # Compute attention scores
        attention = self.attention_fc1(x)
        attention = F.relu(attention)
        attention = self.dropout(attention)
        attention = self.attention_fc2(attention)
        attention = torch.sigmoid(attention)  # [0, 1] weights
        
        # Apply attention (element-wise multiplication)
        attended = x * attention
        
        return attended, attention


class KAN_WithAttention(nn.Module):
    """
    KAN with Feature-Level Attention
    
    Architecture:
    Input -> Feature Attention -> KAN Layers -> Output
    """
    
    def __init__(self, input_dim, hidden_dim=64, grid_size=5,
                 spline_order=3, dropout=0.3, attention_dim=16):
        super(KAN_WithAttention, self).__init__()
        
        self.input_dim = input_dim
        
        # Feature Attention (NEW!)
        self.feature_attention = FeatureAttention(
            input_dim,
            attention_dim=attention_dim,
            dropout=dropout
        )
        
        # KAN layers (same as before)
        self.kan1 = KANLinear(input_dim, hidden_dim, grid_size, spline_order)
        self.kan2 = KANLinear(hidden_dim, hidden_dim // 2, grid_size, spline_order)
        
        # Output
        self.output = nn.Linear(hidden_dim // 2, 1)
        
        # Normalization
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """Forward pass with attention"""
        # Apply feature attention
        x_attended, attention_weights = self.feature_attention(x)
        
        # KAN layers (on attended features)
        x = self.kan1(x_attended)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.kan2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Output
        x = self.output(x)
        x = torch.sigmoid(x)
        
        return x
    
    def get_attention_weights(self, x):
        """Extract attention weights for analysis"""
        self.eval()
        with torch.no_grad():
            if not isinstance(x, torch.Tensor):
                x = torch.FloatTensor(x).to(next(self.parameters()).device)
            _, attention_weights = self.feature_attention(x)
        return attention_weights.cpu().numpy()


print("\n✅ Feature-Level Attention implemented!")

---
## 11. Feature-Level Attention (ÖZGÜN KATKI)

Hafif feature attention mechanism - Her örnek için feature'lara ağırlık verir

In [ ]:
# ============================================================================
# GREY WOLF OPTIMIZER - CPU OPTIMIZED
# ============================================================================

class GreyWolfOptimizer:
    """
    GWO for hyperparameter optimization with multi-metric fitness
    """
    
    def __init__(self, n_wolves, n_iterations, bounds, fitness_func):
        self.n_wolves = n_wolves
        self.n_iterations = n_iterations
        self.bounds = np.array(bounds)
        self.fitness_func = fitness_func
        self.dim = len(bounds)
        
        # Initialize positions
        self.positions = np.random.uniform(
            self.bounds[:, 0],
            self.bounds[:, 1],
            size=(n_wolves, self.dim)
        )
        
        # Alpha, Beta, Delta
        self.alpha_pos = np.zeros(self.dim)
        self.alpha_score = float('-inf')
        self.beta_pos = np.zeros(self.dim)
        self.beta_score = float('-inf')
        self.delta_pos = np.zeros(self.dim)
        self.delta_score = float('-inf')
        
        self.convergence_curve = []
        
    def optimize(self, verbose=True):
        """Run GWO optimization"""
        
        for iteration in range(self.n_iterations):
            # Evaluate all wolves
            for i in range(self.n_wolves):
                fitness = self.fitness_func(self.positions[i])
                
                # Update hierarchy
                if fitness > self.alpha_score:
                    self.delta_score = self.beta_score
                    self.delta_pos = self.beta_pos.copy()
                    self.beta_score = self.alpha_score
                    self.beta_pos = self.alpha_pos.copy()
                    self.alpha_score = fitness
                    self.alpha_pos = self.positions[i].copy()
                elif fitness > self.beta_score:
                    self.delta_score = self.beta_score
                    self.delta_pos = self.beta_pos.copy()
                    self.beta_score = fitness
                    self.beta_pos = self.positions[i].copy()
                elif fitness > self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.positions[i].copy()
            
            # Update parameter a
            a = 2 - iteration * (2.0 / self.n_iterations)
            
            # Update positions
            for i in range(self.n_wolves):
                for j in range(self.dim):
                    r1, r2 = np.random.random(2)
                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha
                    
                    r1, r2 = np.random.random(2)
                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta
                    
                    r1, r2 = np.random.random(2)
                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta
                    
                    self.positions[i, j] = (X1 + X2 + X3) / 3.0
                    self.positions[i, j] = np.clip(
                        self.positions[i, j],
                        self.bounds[j, 0],
                        self.bounds[j, 1]
                    )
            
            self.convergence_curve.append(self.alpha_score)
            
            if verbose and (iteration + 1) % 5 == 0:
                print(f"   Iter {iteration+1:2d}/{self.n_iterations} | Best Fitness: {self.alpha_score:.4f}")
        
        return self.alpha_pos, self.alpha_score, self.convergence_curve


def gwo_kan_fitness_function(params, X_train, y_train, X_val, y_val, 
                             input_dim, config):
    """
    GWO fitness function with multi-metric optimization
    
    Fitness = 0.5*F1 + 0.3*Recall + 0.2*Accuracy
    With accuracy floor guardrail
    """
    # Parse params
    grid_size = int(params[0])
    spline_order = int(params[1])
    hidden_dim = int(params[2])
    learning_rate = params[3]
    
    try:
        # Create model
        model = KAN(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            grid_size=grid_size,
            spline_order=spline_order,
            dropout=0.3
        )
        
        # Train (reduced epochs for speed)
        model, _ = train_kan_model(
            model, X_train, y_train, X_val, y_val,
            learning_rate=learning_rate,
            epochs=30,  # Reduced for GWO speed
            batch_size=config['kan_batch_size'],
            loss_type='Focal',
            patience=10,
            verbose=False
        )
        
        # Find optimal threshold
        model.eval()
        with torch.no_grad():
            X_val_t = torch.FloatTensor(X_val).to(device)
            y_val_proba = model(X_val_t).cpu().numpy().flatten()
        
        threshold_result = find_optimal_threshold(
            y_val, y_val_proba,
            metric='f2',
            threshold_range=config['threshold_range'],
            step=config['threshold_step'],
            accuracy_floor=config['gwo_accuracy_floor'],
            verbose=False
        )
        
        metrics = threshold_result['metrics']
        
        # Calculate fitness
        fitness = (
            config['gwo_fitness_weights']['f1'] * metrics['f1'] +
            config['gwo_fitness_weights']['recall'] * metrics['recall'] +
            config['gwo_fitness_weights']['accuracy'] * metrics['accuracy']
        )
        
        # Apply accuracy floor penalty
        if metrics['accuracy'] < config['gwo_accuracy_floor']:
            fitness *= 0.1  # Heavy penalty
        
        return fitness
    
    except Exception as e:
        print(f"   ⚠️  Fitness eval failed: {e}")
        return 0.0


print("\n✅ GWO implementation ready!")

---
## 10. GWO (Grey Wolf Optimizer) - Multi-Metric Fitness

Optimizing KAN hyperparameters with balanced fitness: 0.5*F1 + 0.3*Recall + 0.2*Accuracy

In [ ]:
# ============================================================================
# KAN TRAINING FUNCTION
# ============================================================================

def train_kan_model(model, X_train, y_train, X_val, y_val,
                   learning_rate=0.01, epochs=100, batch_size=32,
                   loss_type='Focal', patience=15, verbose=True):
    """
    Train KAN model with early stopping based on F2
    
    Parameters:
    -----------
    model : KAN
        KAN model instance
    X_train, y_train : np.ndarray
        Training data
    X_val, y_val : np.ndarray
        Validation data
    learning_rate : float
        Learning rate
    epochs : int
        Maximum epochs
    batch_size : int
        Batch size
    loss_type : str
        'WeightedBCE' or 'Focal'
    patience : int
        Early stopping patience
    verbose : bool
        Print training progress
        
    Returns:
    --------
    tuple
        (trained_model, train_history)
    """
    model = model.to(device)
    
    # Convert to tensors
    X_train_t = torch.FloatTensor(X_train).to(device)
    y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    y_val_t = torch.FloatTensor(y_val).unsqueeze(1).to(device)
    
    # DataLoader
    train_dataset = TensorDataset(X_train_t, y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Loss function
    criterion = get_loss_function(loss_type, y_train)
    criterion = criterion.to(device)
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training history
    history = {'train_loss': [], 'val_f2': [], 'val_loss': []}
    
    # Early stopping
    best_f2 = 0
    patience_counter = 0
    best_model_state = None
    
    if verbose:
        print(f"\n🚀 Training KAN Model (loss={loss_type})...")
        print(f"   Device: {device}")
        print(f"   Epochs: {epochs}, Batch Size: {batch_size}, LR: {learning_rate}")
        print(f"   Early Stopping: patience={patience}, metric=F2\n")
    
    for epoch in range(epochs):
        # Training
        model.train()
        epoch_loss = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        epoch_loss /= len(train_loader)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t)
            val_loss = criterion(val_outputs, y_val_t).item()
            
            val_pred_proba = val_outputs.cpu().numpy().flatten()
            val_pred = (val_pred_proba >= 0.5).astype(int)
            val_f2 = fbeta_score(y_val, val_pred, beta=2, zero_division=0)
        
        # Record history
        history['train_loss'].append(epoch_loss)
        history['val_loss'].append(val_loss)
        history['val_f2'].append(val_f2)
        
        # Early stopping check
        if val_f2 > best_f2:
            best_f2 = val_f2
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        # Print progress
        if verbose and (epoch + 1) % 10 == 0:
            print(f"   Epoch {epoch+1:3d}/{epochs} | Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | Val F2: {val_f2:.4f} | Best F2: {best_f2:.4f}")
        
        # Early stopping
        if patience_counter >= patience:
            if verbose:
                print(f"\n   ⏹️  Early stopping at epoch {epoch+1}")
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    if verbose:
        print(f"\n✅ Training complete! Best Val F2: {best_f2:.4f}")
    
    return model, history


print("\n✅ KAN training function ready!")

In [ ]:
# ============================================================================
# LOSS FUNCTIONS
# ============================================================================

class FocalLoss(nn.Module):
    """
    Focal Loss for imbalanced classification
    
    Parameters:
    -----------
    alpha : float
        Weight for positive class
    gamma : float
        Focusing parameter (higher = more focus on hard examples)
    """
    
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        """Compute focal loss"""
        bce_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()


class WeightedBCELoss(nn.Module):
    """
    Weighted Binary Cross Entropy
    
    Parameters:
    -----------
    pos_weight : float
        Weight for positive class
    """
    
    def __init__(self, pos_weight=1.0):
        super(WeightedBCELoss, self).__init__()
        self.pos_weight = pos_weight
        
    def forward(self, inputs, targets):
        """Compute weighted BCE"""
        # Manual weighted BCE
        loss = - (self.pos_weight * targets * torch.log(inputs + 1e-7) + 
                 (1 - targets) * torch.log(1 - inputs + 1e-7))
        return loss.mean()


def get_loss_function(loss_type, y_train=None):
    """
    Get loss function by name
    
    Parameters:
    -----------
    loss_type : str
        'WeightedBCE' or 'Focal'
    y_train : array-like, optional
        Training labels (for calculating class weights)
        
    Returns:
    --------
    nn.Module
        Loss function
    """
    if loss_type == 'Focal':
        return FocalLoss(alpha=CONFIG['focal_alpha'], gamma=CONFIG['focal_gamma'])
    
    elif loss_type == 'WeightedBCE':
        if y_train is not None:
            # Calculate pos_weight from class distribution
            n_pos = np.sum(y_train == 1)
            n_neg = np.sum(y_train == 0)
            pos_weight = n_neg / n_pos if n_pos > 0 else 1.0
        else:
            pos_weight = 1.0
        
        return WeightedBCELoss(pos_weight=pos_weight)
    
    else:
        return nn.BCELoss()


print("\n✅ Loss functions implemented!")

In [ ]:
# ============================================================================
# KAN (KOLMOGOROV-ARNOLD NETWORK) IMPLEMENTATION
# ============================================================================

class KANLinear(nn.Module):
    """
    KAN Linear Layer with learnable spline functions
    
    Parameters:
    -----------
    in_features : int
        Input dimension
    out_features : int
        Output dimension
    grid_size : int
        Number of grid points for spline
    spline_order : int
        Order of B-spline
    """
    
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        # Learnable grid points
        self.grid = nn.Parameter(
            torch.linspace(-1, 1, grid_size).unsqueeze(0).unsqueeze(0).repeat(
                out_features, in_features, 1
            )
        )
        
        # Learnable spline coefficients
        self.coef = nn.Parameter(
            torch.randn(out_features, in_features, grid_size + spline_order) * 0.1
        )
        
        # Base linear transformation (residual)
        self.base_weight = nn.Parameter(
            torch.randn(out_features, in_features) * 0.1
        )
        
    def b_splines(self, x):
        """Compute B-spline basis functions"""
        batch_size = x.shape[0]
        x = x.unsqueeze(1).unsqueeze(-1)
        grid = self.grid.unsqueeze(0)
        
        distances = torch.abs(x - grid)
        
        basis = torch.zeros(
            batch_size, self.out_features, self.in_features,
            self.grid_size + self.spline_order,
            device=x.device
        )
        
        # RBF-like basis functions
        for i in range(self.grid_size):
            basis[:, :, :, i] = torch.exp(-distances[:, :, :, i] ** 2 / 0.5)
        
        # Polynomial terms
        for i in range(self.spline_order):
            basis[:, :, :, self.grid_size + i] = x.squeeze(-1) ** (i + 1)
        
        return basis
    
    def forward(self, x):
        """Forward pass"""
        basis = self.b_splines(x)
        coef = self.coef.unsqueeze(0)
        
        # Spline output
        spline_output = (basis * coef).sum(dim=-1)
        output = spline_output.sum(dim=-1)
        
        # Add residual
        base_output = torch.matmul(x, self.base_weight.t())
        
        return output + base_output


class KAN(nn.Module):
    """
    KAN for Binary Classification
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    hidden_dim : int
        Hidden layer size
    grid_size : int
        Spline grid size
    spline_order : int
        Spline order
    dropout : float
        Dropout rate
    """
    
    def __init__(self, input_dim, hidden_dim=64, grid_size=5, 
                 spline_order=3, dropout=0.3):
        super(KAN, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        # KAN layers
        self.kan1 = KANLinear(input_dim, hidden_dim, grid_size, spline_order)
        self.kan2 = KANLinear(hidden_dim, hidden_dim // 2, grid_size, spline_order)
        
        # Output layer
        self.output = nn.Linear(hidden_dim // 2, 1)
        
        # Normalization
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """Forward pass"""
        # Layer 1
        x = self.kan1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Layer 2
        x = self.kan2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Output
        x = self.output(x)
        x = torch.sigmoid(x)
        
        return x

print("\n✅ KAN architecture implemented!")

---
## 9. KAN (Kolmogorov-Arnold Network) - Base Implementation

Spline-based activation functions on edges

In [ ]:
# ============================================================================
# BASELINE: RANDOM FOREST WITH CLASS WEIGHTS
# ============================================================================

def train_baseline_rf(X_train, y_train, X_val, y_val, seed):
    """
    Train Random Forest baseline with class balancing
    
    Parameters:
    -----------
    X_train, y_train : np.ndarray
        Training data
    X_val, y_val : np.ndarray
        Validation data
    seed : int
        Random seed
        
    Returns:
    --------
    RandomForestClassifier
        Trained model
    """
    print(f"\n🌲 Training Random Forest Baseline...")
    print(f"   Training samples: {len(y_train)}")
    print(f"   class_weight='balanced'")
    
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        class_weight='balanced',
        random_state=seed,
        n_jobs=-1
    )
    
    rf.fit(X_train, y_train)
    
    # Validation performance
    y_val_pred_proba = rf.predict_proba(X_val)[:, 1]
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    
    val_metrics = calculate_all_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    print(f"\n   Validation Performance (threshold=0.5):")
    print(f"      Recall: {val_metrics['recall']:.4f}")
    print(f"      Precision: {val_metrics['precision']:.4f}")
    print(f"      F1: {val_metrics['f1']:.4f}")
    print(f"      F2: {val_metrics['f2']:.4f}")
    print(f"      Accuracy: {val_metrics['accuracy']:.4f}")
    
    return rf


def evaluate_baseline_with_threshold(rf, X_val, y_val, X_test, y_test, config):
    """
    Evaluate baseline with threshold tuning
    
    Parameters:
    -----------
    rf : RandomForestClassifier
        Trained model
    X_val, y_val : np.ndarray
        Validation data
    X_test, y_test : np.ndarray
        Test data
    config : dict
        Configuration
        
    Returns:
    --------
    dict
        Results with optimal threshold
    """
    # Get probabilities
    y_val_proba = rf.predict_proba(X_val)[:, 1]
    y_test_proba = rf.predict_proba(X_test)[:, 1]
    
    # Find optimal threshold on val
    print(f"\n🔍 Finding optimal threshold on validation set...")
    threshold_result = find_optimal_threshold(
        y_val, 
        y_val_proba,
        metric='f2',
        threshold_range=config['threshold_range'],
        step=config['threshold_step'],
        accuracy_floor=config['gwo_accuracy_floor'],
        verbose=True
    )
    
    optimal_threshold = threshold_result['threshold']
    
    # Evaluate on test with optimal threshold
    print(f"\n🧪 Evaluating on TEST set with threshold={optimal_threshold:.2f}...")
    y_test_pred = (y_test_proba >= optimal_threshold).astype(int)
    test_metrics = calculate_all_metrics(y_test, y_test_pred, y_test_proba)
    
    print(f"\n   📊 Test Performance:")
    print_metrics(test_metrics, prefix="   ")
    
    return {
        'model': rf,
        'optimal_threshold': optimal_threshold,
        'val_metrics': threshold_result['metrics'],
        'test_metrics': test_metrics,
        'threshold_curve': threshold_result['curve']
    }


# ============================================================================
# TRAIN BASELINE FOR ALL DATASETS
# ============================================================================

print("\n" + "="*70)
print("🌲 BASELINE MODELS - RANDOM FOREST")
print("="*70)

baseline_results = {}

for dataset_name, data in prepared_datasets.items():
    print(f"\n{'='*70}")
    print(f"📊 Dataset: {dataset_name}")
    print(f"{'='*70}")
    
    # Train on SMOTE-augmented data
    rf = train_baseline_rf(
        data['X_train_smote'],
        data['y_train_smote'],
        data['X_val'],
        data['y_val'],
        CONFIG['seed']
    )
    
    # Evaluate with threshold tuning
    results = evaluate_baseline_with_threshold(
        rf,
        data['X_val'],
        data['y_val'],
        data['X_test'],
        data['y_test'],
        CONFIG
    )
    
    baseline_results[dataset_name] = results
    
    print(f"\n✅ {dataset_name} Baseline Complete!")
    print(f"   Optimal Threshold: {results['optimal_threshold']:.2f}")
    print(f"   Test F2: {results['test_metrics']['f2']:.4f}")
    print(f"   Test Recall: {results['test_metrics']['recall']:.4f}")
    print(f"   Test Accuracy: {results['test_metrics']['accuracy']:.4f}")

print(f"\n{'='*70}")
print(f"✅ ALL BASELINE MODELS TRAINED")
print(f"{'='*70}")

---
## 🚀 EXECUTION: KAN Base Models (Lightweight)

Hafif KAN modelleri - CPU dostu parametreler